In [25]:
import requests
from tqdm import tqdm
import pandas as pd
# from owlready2 import get_ontology
import os

from standardization_usingsparql import addTaxonColumn, buat_kolom_taxon_awal
from disambiguation_optimized import buat_kamus_kosong, update_kamus_pake_wikidata, update_df_pake_kamus, update_df_pake_path_ujung, removeOtherThanNCBI
from preprocess import cleaning, splitInteractionToNodeEdge
from filterNodeEdge import removeNodeAndEdgeByFilter,takeNodeAndEdgeByFilter,removeEdgesNotInNodes
from handler_umum import contains_string_entire_column
from vectorReferenced import get_taxon_vector,cek_ncbi_id_by_wiki_id_via_string

In [26]:
#1 inisiasi
# Ambil NCBI OBO
# ncbi_link = os.getcwd()+'/../data/ncbitaxon.owl'
# onto = get_ontology(ncbi_link)
# onto.load()
# obo = onto.get_namespace('http://purl.obolibrary.org/obo/')
#parameter
virus_txt = 'Cucumber mosaic virus'.replace(' ','%20')
nama_file = '1cucu'
tipe_interaksi_virus = 'hasHost' #pathogenOf, pake relasi hasHost lebih dapat banyak relasi dari pada pathogenOf
tipe_interaksi_tanaman = 'hostOf' #hasPathogen, pake relasi hostOf lebih dapat banyak relasi dari pada hasPathogen
tipe_interaksi_serangga_ke_tanaman = 'hasHost' 
tipe_interaksi_serangga_ke_virus = 'hostOf' 
ncbi_server_url = 'http://localhost:3030/mydataset/query'

In [27]:
virus_search = get_taxon_vector(virus_txt,ncbi_server_url)
virus_search

[('spesies', 'NCBI:12305_Cucumber mosaic virus'),
 ('genus', 'NCBI:12304_Cucumovirus'),
 ('famili', 'NCBI:39740_Bromoviridae'),
 ('ordo', 'NCBI:2732544_Martellivirales'),
 ('kelas', 'NCBI:2732461_Alsuviricetes'),
 ('filum', 'NCBI:2732406_Kitrinoviricota'),
 ('kingdom', 'NCBI:2732396_Orthornavirae'),
 ('superkingdom', 'NCBI:10239_Viruses')]

In [28]:
get_taxon_vector("Myzus",ncbi_server_url)

[('genus', 'NCBI:13163_Myzus'),
 ('famili', 'NCBI:27482_Aphididae'),
 ('ordo', 'NCBI:7524_Hemiptera'),
 ('kelas', 'NCBI:50557_Insecta'),
 ('filum', 'NCBI:6656_Arthropoda'),
 ('kingdom', 'NCBI:33208_Metazoa'),
 ('superkingdom', 'NCBI:2759_Eukaryota')]

In [29]:
virus_search[1][1].split('_')

['NCBI:12304', 'Cucumovirus']

In [30]:
#2 ambil data dari globi
kolom=[
    'source_taxon_external_id',
    'source_taxon_name',
    'source_taxon_path',
    'source_taxon_path_ids',
    'source_taxon_path_ranks',
    
    'interaction_type',
    
    'target_taxon_external_id',
    'target_taxon_name',
    'target_taxon_path',
    'target_taxon_path_ids',
    'target_taxon_path_ranks',
]

interactionType=tipe_interaksi_virus
link="https://api.globalbioticinteractions.org/interaction?sourceTaxon="+virus_search[0][1].split('_')[0]+"&interactionType="+interactionType+"&fields="+(','.join(kolom))
response = requests.get(link)
res=response.json()

if not res['data']:
    print('kosong')
# JSON To Pandas Dataframe
df = pd.json_normalize(res, record_path =['data'])

link="https://api.globalbioticinteractions.org/interaction?sourceTaxon="+virus_search[1][1].split('_')[0]+"&interactionType="+interactionType+"&fields="+(','.join(kolom))
response = requests.get(link)
res=response.json()
if not res['data']:
    print('kosong')
# JSON To Pandas Dataframe
df_ = pd.json_normalize(res, record_path =['data'])
df=pd.concat([df,df_])

df.columns = kolom




In [31]:
# contains_string_entire_column(df,'Insect')
df

,source_taxon_external_id,source_taxon_name,source_taxon_path,source_taxon_path_ids,source_taxon_path_ranks,interaction_type,target_taxon_external_id,target_taxon_name,target_taxon_path,target_taxon_path_ids,target_taxon_path_ranks
0,INAT_TAXON:973842,Cucumber mosaic virus,Viruses | Kitrinoviricota | Alsuviricetes | Ma...,INAT_TAXON:131236 | INAT_TAXON:1123684 | INAT_...,kingdom | phylum | class | order | family | ge...,hasHost,INAT_TAXON:363481,Scaevola taccada,Plantae | Tracheophyta | Magnoliophyta | Magno...,INAT_TAXON:47126 | INAT_TAXON:211194 | INAT_TA...,kingdom | subkingdom | phylum | class | order ...
1,INAT_TAXON:973842,Cucumber mosaic virus,Viruses | Kitrinoviricota | Alsuviricetes | Ma...,INAT_TAXON:131236 | INAT_TAXON:1123684 | INAT_...,kingdom | phylum | class | order | family | ge...,hasHost,GBIF:1944973,Viola,Animalia | Arthropoda | Insecta | Lepidoptera ...,GBIF:1 | GBIF:54 | GBIF:216 | GBIF:797 | GBIF:...,kingdom | phylum | class | order | family | genus
2,INAT_TAXON:973842,Cucumber mosaic virus,Viruses | Kitrinoviricota | Alsuviricetes | Ma...,INAT_TAXON:131236 | INAT_TAXON:1123684 | INAT_...,kingdom | phylum | class | order | family | ge...,hasHost,INAT_TAXON:127010,Nandina domestica,Plantae | Tracheophyta | Magnoliophyta | Magno...,INAT_TAXON:47126 | INAT_TAXON:211194 | INAT_TA...,kingdom | subkingdom | phylum | class | order ...
3,INAT_TAXON:973842,Cucumber mosaic virus,Viruses | Kitrinoviricota | Alsuviricetes | Ma...,INAT_TAXON:131236 | INAT_TAXON:1123684 | INAT_...,kingdom | phylum | class | order | family | ge...,hasHost,INAT_TAXON:127010,Nandina domestica,Plantae | Tracheophyta | Magnoliophyta | Magno...,INAT_TAXON:47126 | INAT_TAXON:211194 | INAT_TA...,kingdom | subkingdom | phylum | class | order ...
4,INAT_TAXON:973842,Cucumber mosaic virus,Viruses | Kitrinoviricota | Alsuviricetes | Ma...,INAT_TAXON:131236 | INAT_TAXON:1123684 | INAT_...,kingdom | phylum | class | order | family | ge...,hasHost,INAT_TAXON:363481,Scaevola taccada,Plantae | Tracheophyta | Magnoliophyta | Magno...,INAT_TAXON:47126 | INAT_TAXON:211194 | INAT_TA...,kingdom | subkingdom | phylum | class | order ...
...,...,...,...,...,...,...,...,...,...,...,...
89,GBIF:8765348,Cucumovirus,| Bromoviridae | Cucumovirus |,GBIF:8 | GBIF:6226 | GBIF:9155085 | GBIF:8765348,kingdom | family | genus | species,pathogenOf,GBIF:2874172,Passiflora,Plantae | Tracheophyta | Magnoliopsida | Malpi...,GBIF:6 | GBIF:7707728 | GBIF:220 | GBIF:1414 |...,kingdom | phylum | class | order | family | genus
90,GBIF:8765348,Cucumovirus,| Bromoviridae | Cucumovirus |,GBIF:8 | GBIF:6226 | GBIF:9155085 | GBIF:8765348,kingdom | family | genus | species,pathogenOf,EOL:468029,Helianthus tuberosus,Plantae | Tracheophyta | Magnoliopsida | Aster...,EOL_V2:281 | EOL:4077 | EOL:283 | EOL:4205 | E...,kingdom | phylum | class | order | family | ge...
91,GBIF:8765348,Cucumovirus,| Bromoviridae | Cucumovirus |,GBIF:8 | GBIF:6226 | GBIF:9155085 | GBIF:8765348,kingdom | family | genus | species,pathogenOf,GBIF:6,Plantae,Plantae,GBIF:6,kingdom
92,GBIF:8765348,Cucumovirus,| Bromoviridae | Cucumovirus |,GBIF:8 | GBIF:6226 | GBIF:9155085 | GBIF:8765348,kingdom | family | genus | species,pathogenOf,GBIF:5290052,Zea mays,Plantae | Tracheophyta | Liliopsida | Poales |...,GBIF:6 | GBIF:7707728 | GBIF:196 | GBIF:1369 |...,kingdom | phylum | class | order | family | ge...


In [32]:
#3 cleaning
df=cleaning(df, dropNonNCBI=False)

drop duplicate
sebelum :  168
setelah:  47
drop tidak mengandung NCBI dan taxon pathnya null
sebelum :  47
setelah:  46
drop duplicate nama sama beda id
sebelum :  46
setelah:  45


In [33]:
# drop duplicate
# sebelum :  94
# setelah:  47
# drop tidak mengandung NCBI dan taxon pathnya null
# sebelum :  47
# setelah:  46
# drop duplicate nama sama beda id
# sebelum :  46
# setelah:  45

In [34]:
#4 splitting layer 1 interaksi virus
df_node, df_edge = splitInteractionToNodeEdge(df)

In [35]:
len(df_node),len(df_edge)

(48, 45)

In [36]:
#5 disambiguasi layer 1 interaksi virus
kamus_ncbi = buat_kamus_kosong(df_node)
kamus_ncbi = update_kamus_pake_wikidata(kamus_ncbi)
#update dataframe pake kamus
df_node,df_edge = update_df_pake_kamus(kamus_ncbi,df_node,df_edge)
df_node,df_edge = update_df_pake_path_ujung(df_node,df_edge)

['INAT_TAXON', 'GBIF', 'EOL_V2', 'EOL'] 4  database,  4  kali perulangan akses NCBI
INAT_TAXON : jumlah id 12


100%|██████████| 11/11 [00:00<00:00, 290171.97it/s]


GBIF : jumlah id 73


100%|██████████| 67/67 [00:00<00:00, 449629.39it/s]

EOL_V2 : jumlah id 2



100%|██████████| 1/1 [00:00<00:00, 32263.88it/s]


EOL : jumlah id 22


100%|██████████| 20/20 [00:00<00:00, 307275.02it/s]


In [37]:
#untuk laporan
#kode database
print(kamus_ncbi.keys())

key=[]
val=[]
for i in kamus_ncbi:
    key.extend(list(kamus_ncbi[i].keys()))
    val.extend(list(kamus_ncbi[i].values()))
    
df_kamus=pd.DataFrame({'key':key,'val':val})

# csv
# df_kamus.to_csv('output.xlsx', index=False)

# semua 
# df_kamus
# yang kosong
df_kamus[(df_kamus.val != '')].count()

dict_keys(['INAT_TAXON', 'GBIF', 'EOL_V2', 'EOL'])


key    94
val    94
dtype: int64

In [38]:
#6 standarisasi layer 1 interaksi virus
df_node = buat_kolom_taxon_awal(df_node) #buat kolom taxon, default none
df_node = addTaxonColumn(df_node,'http://localhost:3030/mydataset/query') # isi pake ncbi
df_node, df_edge = removeOtherThanNCBI(df_node,df_edge)# Hapus kalo masih ada selain NCBI
df_edge = removeEdgesNotInNodes(df_node, df_edge) #hapus edge yang tidak ada nodenya

{'GBIF'}
------------------------------------
node
sebelum  48
sesudah  47
edge
target: sebelum 45
target: sesudah 44
source: sebelum 45
source: sesudah 45


In [39]:
#7 praproses interaksi tanaman
df_plant=df_node[df_node.kingdom=='NCBI:33090_Viridiplantae']
interactionType=tipe_interaksi_tanaman
node_to_add=pd.DataFrame(columns = [
    'taxon_id',
    'taxon_name',
    'taxon_path',
    'taxon_path_ids',
    'taxon_path_rank',
])
edge_to_add=pd.DataFrame(columns = [
    'source_taxon_id',
    'target_taxon_id',
    'interaction_type',
])
kebalikan={
    'hostOf':'hasHost',
    'hasPathogen':'pathogenOf', 
    'pollinatedBy':'pollinates', 
    'flowersVisitedBy':'visitFlowersOf',
    'visitedBy':'visit'
}
#tanaman
for idx,i in tqdm(df_plant.iterrows(), total=df_plant.shape[0]):
    plant=i.taxon_name.replace(' ','%20')
    link="https://api.globalbioticinteractions.org/interaction?sourceTaxon="+plant+"&interactionType="+interactionType+"&fields="+(','.join(kolom))
    response = requests.get(link)
    res=response.json()
    #list data
    taxon_id=[]
    taxon_name=[]
    taxon_path=[]
    taxon_path_ids=[]
    taxon_path_rank=[]
    interaction_type=[]
    for x in res['data']:
        taxon_id.append(x[6])
        taxon_name.append(x[7])
        taxon_path.append(x[8])
        taxon_path_ids.append(x[9])
        taxon_path_rank.append(x[10])
        if x[5] in ['hostOf', 'hasPathogen', 'pollinatedBy', 'flowersVisitedBy','visitedBy']:
            interaction_type.append(kebalikan[x[5]])
        else:
            interaction_type.append('patogennya')
    ## NODE
    #concat ke dataframe lama
    node_to_add = pd.concat([
        node_to_add,
        pd.DataFrame({
            'taxon_id':taxon_id,
            'taxon_name':taxon_name,
            'taxon_path':taxon_path,
            'taxon_path_ids':taxon_path_ids,
            'taxon_path_rank':taxon_path_rank,
        })
    ], ignore_index = True)
    node_to_add.reset_index()
    ## EDGE
    #concat ke dataframe lama
    edge_to_add = pd.concat([
        edge_to_add,
        pd.DataFrame({
            'source_taxon_id': taxon_id, # patogen
            'target_taxon_id': i.taxon_id, # tanaman
            'interaction_type': interaction_type})
    ], ignore_index = True)
    edge_to_add.reset_index()

100%|██████████| 38/38 [00:46<00:00,  1.23s/it]


In [40]:
# 7 praproses interaksi serangga ke tanaman
df_insect = df_node[df_node['class']=='NCBI:50557_Insecta']
interactionType = tipe_interaksi_serangga_ke_tanaman
#serangga
for idx,i in tqdm(df_insect.iterrows(), total=df_insect.shape[0]):
    insect=i.taxon_name.replace(' ','%20')
    link="https://api.globalbioticinteractions.org/interaction?sourceTaxon="+insect+"&interactionType="+interactionType+"&fields="+(','.join(kolom))
    response = requests.get(link)
    res=response.json()
    #list data
    taxon_id=[]
    taxon_name=[]
    taxon_path=[]
    taxon_path_ids=[]
    taxon_path_rank=[]
    interaction_type=[]
    for x in res['data']:
        taxon_id.append(x[6])
        taxon_name.append(x[7])
        taxon_path.append(x[8])
        taxon_path_ids.append(x[9])
        taxon_path_rank.append(x[10])
        interaction_type.append(x[5])
    ## NODE
    #concat ke dataframe lama
    node_to_add = pd.concat([
        node_to_add,
        pd.DataFrame({
            'taxon_id':taxon_id,
            'taxon_name':taxon_name,
            'taxon_path':taxon_path,
            'taxon_path_ids':taxon_path_ids,
            'taxon_path_rank':taxon_path_rank,
        })
    ], ignore_index = True)
    node_to_add.reset_index()
    ## EDGE
    #concat ke dataframe lama
    edge_to_add = pd.concat([
        edge_to_add,
        pd.DataFrame({
            'source_taxon_id': i.taxon_id, # serangga
            'target_taxon_id': taxon_id, # inagnya
            'interaction_type': interaction_type
            })
    ], ignore_index = True)
    edge_to_add.reset_index()

100%|██████████| 2/2 [00:01<00:00,  1.23it/s]


In [41]:
# # 7 praproses interaksi serangga ke virus
# df_insect = df_node[df_node['class']=='NCBI:50557_Insecta']
# interactionType = tipe_interaksi_serangga_ke_virus
# #serangga
# for idx,i in tqdm(df_insect.iterrows(), total=df_insect.shape[0]):
#     insect=i.taxon_name.replace(' ','%20')
#     link="https://api.globalbioticinteractions.org/interaction?sourceTaxon="+insect+"&interactionType="+interactionType+"&fields="+(','.join(kolom))
#     response = requests.get(link)
#     res=response.json()
#     #list data
#     taxon_id=[]
#     taxon_name=[]
#     taxon_path=[]
#     taxon_path_ids=[]
#     taxon_path_rank=[]
#     interaction_type=[]
#     for x in res['data']:
#         taxon_id.append(x[6])
#         taxon_name.append(x[7])
#         taxon_path.append(x[8])
#         taxon_path_ids.append(x[9])
#         taxon_path_rank.append(x[10])
#         interaction_type.append(x[5])
#     ## NODE
#     #concat ke dataframe lama
#     node_to_add = pd.concat([
#         node_to_add,
#         pd.DataFrame({
#             'taxon_id':taxon_id,
#             'taxon_name':taxon_name,
#             'taxon_path':taxon_path,
#             'taxon_path_ids':taxon_path_ids,
#             'taxon_path_rank':taxon_path_rank,
#         })
#     ], ignore_index = True)
#     node_to_add.reset_index()
#     ## EDGE
#     #concat ke dataframe lama
#     edge_to_add = pd.concat([
#         edge_to_add,
#         pd.DataFrame({
#             'source_taxon_id': i.taxon_id, # serangga
#             'target_taxon_id': taxon_id, # inagnya
#             'interaction_type': interaction_type
#             })
#     ], ignore_index = True)
#     edge_to_add.reset_index()

In [42]:
print(len(node_to_add),len(edge_to_add))

19919 19919


In [43]:
#8 cleaning layer 2 interaksi tanaman dan serangga
node_to_add.drop_duplicates(inplace=True)
hapus=[i for i,d in node_to_add[
    (node_to_add.taxon_id.str.contains('NCBI')==False) & (node_to_add.taxon_path_ids.isnull())
].iterrows()]
node_to_add.drop(hapus, inplace=True)
edge_to_add.drop_duplicates(inplace=True)

In [377]:
#untuk laporan
print(len(node_to_add),len(edge_to_add))
print(len(df_node)+len(node_to_add),len(df_edge)+len(edge_to_add))

1863 3282
1908 3326


In [378]:
# 9 disambiguasi dan standarisasi interaksi tanaman
kamus_ncbi = buat_kamus_kosong(node_to_add)
kamus_ncbi = update_kamus_pake_wikidata(kamus_ncbi)
#update dataframe pake kamus
node_to_add,edge_to_add = update_df_pake_kamus(kamus_ncbi,node_to_add,edge_to_add)
node_to_add,edge_to_add = update_df_pake_path_ujung(node_to_add, edge_to_add)

# 17+219+48+205+2

['GBIF', 'EOL', 'EOL_V2', 'IRMNG', 'WD', 'IF', 'NBN', 'WORMS', 'INAT_TAXON', 'ALATaxon'] 10  database,  10  kali perulangan akses NCBI
GBIF : jumlah id 952
GBIF : query terlalu panjang, dilakukan chunk


100%|██████████| 37/37 [00:00<00:00, 233016.89it/s]


EOL : jumlah id 738
EOL : query terlalu panjang, dilakukan chunk


100%|██████████| 75/75 [00:00<00:00, 459901.75it/s]


EOL_V2 : jumlah id 200
EOL_V2 : query terlalu panjang, dilakukan chunk


...

IF : tidak diketahui predikatnya
NBN : jumlah id 108


100%|██████████| 79/79 [00:00<00:00, 424263.78it/s]


WORMS : tidak diketahui predikatnya
INAT_TAXON : jumlah id 7


100%|██████████| 7/7 [00:00<00:00, 73584.28it/s]


ALATaxon : tidak diketahui predikatnya


In [379]:
#untuk laporan
#kode database
print(kamus_ncbi.keys())

key=[]
val=[]
for i in kamus_ncbi:
    key.extend(list(kamus_ncbi[i].keys()))
    val.extend(list(kamus_ncbi[i].values()))
    
df_kamus=pd.DataFrame({'key':key,'val':val})

# csv
# df_kamus.to_csv('output.xlsx', index=False)

# semua 
print(df_kamus.count())
# yang kosong
# df_kamus[(df_kamus.val != '') & (df_kamus.key.str.contains("NBN"))]
print(df_kamus[(df_kamus.val != '')].count())

dict_keys(['GBIF', 'EOL', 'EOL_V2', 'IRMNG', 'WD', 'IF', 'NBN', 'WORMS', 'INAT_TAXON', 'ALATaxon'])
key    2110
val    2110
dtype: int64
key    1366
val    1366
dtype: int64


In [380]:
# node_to_add[node_to_add["taxon_id"].str.contains("NCBI") == False]

In [381]:
#10 praproses terakhir
node_to_add = buat_kolom_taxon_awal(node_to_add) #buat kolom taxon, isi none dan isi dari path
node_to_add = addTaxonColumn(node_to_add,'http://localhost:3030/mydataset/query') #isi kolom taxon, pake NCBI
node_to_add,edge_to_add = removeOtherThanNCBI(node_to_add,edge_to_add) #hapus selain NCBI
print(len(node_to_add),len(edge_to_add))#untuk laporan
# concat dengan data lama
df_node=pd.concat([df_node,node_to_add], axis=0)
df_edge=pd.concat([df_edge,edge_to_add], axis=0)
df_node.drop_duplicates(subset=["taxon_id"], keep='last',inplace=True)#hapus duplikasi node
df_edge = removeEdgesNotInNodes(df_node, df_edge)#edge menyesuaikan
df_node,df_edge = removeNodeAndEdgeByFilter(df_node[df_node.kingdom.isnull()], df_node,df_edge) # hapus kingdom isi null
df_edge = removeEdgesNotInNodes(df_node, df_edge) #hapus lagi edge kalo masih ada edge tidak ada di node
# Menghapus kolom yang tidak lagi dibutuhkan
if 'taxon_rank' in node_to_add.columns:
    node_to_add.drop(['taxon_rank'], axis=1,inplace=True) #dihapus karena menyesuaikan draft saja. dan karena tidak tahu mbken apa

{'WORMS', 'IF', 'EOL_V2', 'NBN', 'EOL', 'ALATaxon'}
------------------------------------
node
sebelum  1863
sesudah  1837
edge
target: sebelum 3282
target: sesudah 3282
source: sebelum 3282
source: sesudah 3178
1837 3178


In [382]:
#untuk laporan
len(df_node),len(df_edge)

(1383, 2789)

In [383]:
# reset index
df_node.reset_index(drop=True,inplace=True)
df_edge.reset_index(drop=True,inplace=True)

In [ ]:
#11 praproses tambahan
#isi data kosong. mengisi takson kosong, dengan takson sebelumnya, untuk tambalan
takson=[
    'superkingdom','kingdom','phylum','class','order','family','genus','species'
]

for x,i in enumerate(takson):
    if (i!='superkingdom'): #selain superkingdom update dengan data sebelumnya
        for idx, row in df_node[pd.isnull(df_node[i])].iterrows():
            df_node.loc[idx,[i]] = row[takson[x-1]]+'^'+i
    else: 
        for idx, row in df_node[pd.isnull(df_node[i])].iterrows():
            df_node.loc[idx,[i]] = row[takson[x+1]]+'^'+i

In [ ]:
#12 praproses tambahan
# pengelompokan
# Binning of the data based on serangga, virus, tanaman, nogroup
filter_tanaman = df_node['kingdom']=='NCBI:33090_Viridiplantae' 
filter_virus = (
    (df_node['superkingdom']=='NCBI:10239_Viruses')
    | (df_node.taxon_name.str.contains(r'\b(virus\w*|\w*virus)\b',case=False))
    | (df_node.taxon_path.str.contains(r'\b(virus\w*|\w*virus)\b', case=False)) 
    #jika berawalan atau berakhiran kata virus
)
filter_serangga = ((df_node['class']=='NCBI:50557_Insecta') )#& (df_node['order']!='NCBI:7399_Hymenoptera')) #dan bukan lebah hymenoptera
# df_node['kingdom']=='NCBI:33208_Metazoa' and (df_node['phylum']=='NCBI:6656_Arthropoda' and 
#ini jadi dilema, ada kalau ingin memasukan artropoda lain misal, laba2 maka pake or. untuk saat ini fokus ke insect

df_node.loc[filter_tanaman, ['group','color']] = ["tanaman",'#1f922b'] #hijau
df_node.loc[filter_virus, ['group','color']] = ['virus','#671f92'] #ungu
df_node.loc[filter_serangga, ['group','color']] = ['serangga','#b22222'] #merah
df_node.loc[(
    (filter_tanaman==False) & 
    (filter_virus==False) &
    (filter_serangga==False) 
    ),['group','color']] = ['nogroup','#EADDCA'] #abu-abu

In [384]:
# akhir pra proses
# save file
# df_node.to_csv(os.getcwd()+'/dari_praproses/'+nama_file+'_node.csv')
# df_edge.to_csv(os.getcwd()+'/dari_praproses/'+nama_file+'_edge.csv')

Dibawah ini tidak masuk pra proses